In [ ]:
import xarray as xr
import geopandas as gpd
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import requests
import pypsa
import shutil
from rasterio.plot import show
from shapely.wkt import loads
from shapely.geometry import Point
from shapely.geometry import LineString
from pathlib import Path
import country_converter as coco


# change current directory to parent folder
import os
import sys

if not os.path.isdir("pypsa-earth"):
    os.chdir("../..")
sys.path.append(os.getcwd()+"/pypsa-earth/scripts")

from _helpers import read_csv_nafix
from _helpers import two_digits_2_name_country, country_name_2_two_digits

In [ ]:
cc = coco.CountryConverter()

In [ ]:
import pandas as pd

df_list = []

for scenario in Path("pypsa-earth/results").glob("*"):
    if scenario.is_dir():
        stats_path = Path.joinpath(scenario, "stats.csv")
        if stats_path.is_file():
            df_list.append(read_csv_nafix(str(stats_path), header=[0, 1], index_col=0))
        else:
            print("Missing entry for " + str(stats_path))

stats = pd.concat(df_list)
stats.to_csv("documentation/stats_merged.csv")

In [ ]:
stats = stats[stats.index.str.len() <= 2]

In [ ]:
cc = coco.CountryConverter()

In [ ]:
cc = coco.CountryConverter()
UN_2_code = coco.convert(cc.UN.name_short, to="ISO2")
missing_codes = set(UN_2_code) - set(stats.index)
missing_codes_name_short = coco.convert(list(missing_codes), to="name_short")
dict(zip(missing_codes, missing_codes_name_short))

In [ ]:
UN_stats = stats.loc[stats.index.intersection(UN_2_code)]

total_pop = UN_stats[('build_shapes', 'pop')].sum()

print(("Total population: {:.2f}".format(total_pop/1e9)))

total_pop_working = UN_stats.loc[UN_stats[('snakemake_status', 'solve_network')], ('build_shapes', 'pop')].sum()

print(("Total population working: {:.2f}".format(total_pop_working/1e9)))
print("Working represented country [%]: {:.2f}%%".format(total_pop_working/total_pop*100))

In [ ]:
UN_stats[('snakemake_status', 'solve_network')]

In [ ]:
UN_stats.loc[UN_stats[('snakemake_status', 'solve_network')]]

In [ ]:
UN_stats.loc[~UN_stats[('snakemake_status', 'solve_network')], ('build_shapes', 'pop')].sort_values(ascending=False).index

In [ ]:
# summary_scenarios = "pypsa-earth/test.csv" #"pypsa-earth/scenarios/stats.csv"

#### Read the statistics of the workflow

In [ ]:
# stats = read_csv_nafix(summary_scenarios, header=[0, 1], index_col=0)

## Plot the status of the workflow by country

Collect in the column status_level, the number of rules that have succeded

In [ ]:
stats

In [ ]:
stats["status_level"] = stats[("snakemake_status", "solve_network")]
stats["status_level"].sum()/stats.shape[0]

In [ ]:

import cartopy
import cartopy.io.shapereader as shpreader
import cartopy.crs as ccrs
import cartopy.feature as cf
import matplotlib.cm as cm

import country_converter as coco

stats["iso_a3"] = coco.convert(names=stats.index, to='ISO3')

def custom_cmap(x):

    if x[("snakemake_status", "solve_network")] == True:  # till solved network
        return (0, 1, 0)  # green
    if x[("snakemake_status", "simplify_network")] == True:  #
        return (0, .5, 0)  # dark green
    
    return (1.0, 0.8, 0)  # orange
    # if x[("snakemake_status", "base_network")] == True:
    #     return (1, 1, 0)  # yellow
    # if x[("snakemake_status", "build_shapes")] == True:
    #     return (1.0, 0.6, 0)  # orange
    # return (1, 0, 0)  # red

plt.figure(figsize=(5, 3), dpi=300)
ax = plt.axes(projection=ccrs.PlateCarree())
#ax.add_feature(cf.LAND)
#ax.add_feature(cf.OCEAN)
ax.add_feature(cf.COASTLINE)
# ax.add_feature(cf.BORDERS, linestyle='-', alpha=.5)
#ax.add_feature(cf.LAKES, alpha=0.95)
#ax.add_feature(cf.RIVERS)
ax.add_feature(cf.BORDERS)
# ax.set_extent([-150, 60, -25, 60])

countries = gpd.read_file(
               gpd.datasets.get_path("naturalearth_lowres")).set_index("iso_a3")

stats_plot = stats.set_index("iso_a3")

exclude_countries = [] #["USA", "CAN", "MEX", "RUS", "GRL",  "TLS"] #"IDN",
exclude_continents = [] #["Europe", "Oceania"]

count_countries = 0

for c_iso_a3, row in countries.iterrows():
    # if c_iso_a3 in exclude_countries or coco.convert(c_iso_a3, to="Continent") in exclude_continents:
    #     continue
    if c_iso_a3 in stats_plot.index:
        # if not stats_plot.loc[c_iso_a3,("snakemake_status", "solve_network")]:
        #     continue
        ax.add_geometries(row["geometry"], ccrs.PlateCarree(),
                          facecolor=custom_cmap(stats_plot.loc[c_iso_a3]),
                          label=c_iso_a3)
        count_countries += stats_plot.loc[c_iso_a3, ("snakemake_status", "solve_network")]
        # if stats.loc[country.attributes['ISO_A2'], ("snakemake_status", "solve_network")]:
        #     ax.add_geometries(country.geometry, ccrs.PlateCarree(),
        #                     facecolor=(0, 1, 0),
        #                     label=country.attributes['ISO_A2'])


plt.savefig("documentation/img.png")
plt.show()

In [ ]:

# import cartopy
# import cartopy.io.shapereader as shpreader
# import cartopy.crs as ccrs
# import cartopy.feature as cf
# import matplotlib.cm as cm

# def custom_cmap(x):
#     if x[("snakemake_status", "solve_network")] == True:  # till solved network
#         return (0, 1, 0)  # green
    
#     print("Non-green element: " + str(x.name))

#     if x[("snakemake_status", "simplify_network")] == True:  #
#         return (0, .5, 0)  # dark green
#     if x[("snakemake_status", "base_network")] == True:
#         return (1, 1, 0)  # yellow
#     if x[("snakemake_status", "build_shapes")] == True:
#         return (1.0, 0.6, 0)  # orange
#     return (1, 0, 0)  # red

# plt.figure(figsize=(5, 3), dpi=300)
# ax = plt.axes(projection=ccrs.PlateCarree())
# #ax.add_feature(cf.LAND)
# #ax.add_feature(cf.OCEAN)
# ax.add_feature(cf.COASTLINE)
# # ax.add_feature(cf.BORDERS, linestyle='-', alpha=.5)
# #ax.add_feature(cf.LAKES, alpha=0.95)
# #ax.add_feature(cf.RIVERS)
# ax.add_feature(cf.BORDERS)
# # ax.set_extent([-150, 60, -25, 60])

# shpfilename = shpreader.natural_earth(resolution='110m',
#                                       category='cultural',
#                                       name='admin_0_countries')
# reader = shpreader.Reader(shpfilename)
# countries = reader.records()

# for country in countries:
#     if country.attributes['ISO_A2'] in stats.index :
#         ax.add_geometries(country.geometry, ccrs.PlateCarree(),
#                           facecolor=custom_cmap(stats.loc[country.attributes['ISO_A2']]),
#                           label=country.attributes['ISO_A2'])
#         # if stats.loc[country.attributes['ISO_A2'], ("snakemake_status", "solve_network")]:
#         #     ax.add_geometries(country.geometry, ccrs.PlateCarree(),
#         #                     facecolor=(0, 1, 0),
#         #                     label=country.attributes['ISO_A2'])


# plt.savefig("documentation/img.png")
# plt.show()

In [ ]:
"FRA" in countries.index

In [ ]:
"FRA" in stats_plot.index

In [ ]:
# fix 

In [ ]:

import cartopy
import cartopy.io.shapereader as shpreader
import cartopy.crs as ccrs
import cartopy.feature as cf
import matplotlib.cm as cm

def custom_cmap_original(x):
    if x == "NG":  # till solved network
        return (0, 1, 0)  # green
    return (0.2, 0.2, 1)  # blue

plt.figure(figsize=(5, 3), dpi=300)
ax = plt.axes(projection=ccrs.PlateCarree())
#ax.add_feature(cf.LAND)
#ax.add_feature(cf.OCEAN)
ax.add_feature(cf.COASTLINE)
# ax.add_feature(cf.BORDERS, linestyle='-', alpha=.5)
#ax.add_feature(cf.LAKES, alpha=0.95)
#ax.add_feature(cf.RIVERS)
ax.add_feature(cf.BORDERS)
# ax.set_extent([-150, 60, -25, 60])

shpfilename = shpreader.natural_earth(resolution='110m',
                                      category='cultural',
                                      name='admin_0_countries')
reader = shpreader.Reader(shpfilename)
countries = reader.records()

for country in countries:
    if country.attributes['CONTINENT'] == "Africa":
        ax.add_geometries(country.geometry, ccrs.PlateCarree(),
                          facecolor=custom_cmap_original(country.attributes['ISO_A2']),
                          label=country.attributes['ISO_A2'])


plt.savefig("documentation/img_original.png")
plt.show()

In [ ]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

In [ ]:
world.keys()

In [ ]:
plt.figure(figsize=(5, 3), dpi=300)
ax = plt.axes(projection=ccrs.PlateCarree())
#ax.add_feature(cf.LAND)
#ax.add_feature(cf.OCEAN)
ax.add_feature(cf.COASTLINE)
# ax.add_feature(cf.BORDERS, linestyle='-', alpha=.5)
#ax.add_feature(cf.LAKES, alpha=0.95)
#ax.add_feature(cf.RIVERS)
ax.add_feature(cf.BORDERS)
# ax.set_extent([-150, 60, -25, 60])

shpfilename = shpreader.natural_earth(resolution='110m',
                                      category='cultural',
                                      name='admin_0_countries')
reader = shpreader.Reader(shpfilename)
countries = reader.records()

for country in countries:
    if country.attributes['ISO_A2'] in stats[stats[("snakemake_status", "add_electricity")] == False].index:
        print(country.attributes['ISO_A2'])
        ax.add_geometries(country.geometry, ccrs.PlateCarree(),
                          facecolor=(1, 0, 0),
                          label=country.attributes['ISO_A2'])


plt.savefig("documentation/img_not_working.png")
plt.show()

In [ ]:
missing_rules_by_country = n_rules - stats.loc[stats["status_level"] < n_rules, "status_level"]
missing_rules_by_country

In [ ]:
for country in countries:
    print(country.attributes.keys())
    break;

In [ ]:
import pandas as pd

df_list = []

for c in stats.index[stats.loc[:, ("snakemake_status", "solve_network")]==False]:

    if c not in UN_2_code:
        continue
    stat_path = os.getcwd() + f"/pypsa-earth/results/{c}/stats.csv"
    # # print(stat_path)
    # if not os.path.isfile(stat_path):
    #     print(f"SKIP {c}")
    #     # continue
    
    # os.remove(os.getcwd() + f"/pypsa-earth/results/{c}/scenario.done")
    # os.remove(os.getcwd() + f"/pypsa-earth/results/{c}/stats.csv")

    for d in ["results", "resources", "networks"]:
        dirpath = os.getcwd() + f"/pypsa-earth/{d}/{c}/"
        if os.path.isdir(dirpath):
            # print(dirpath)
            shutil.rmtree(dirpath)

In [ ]:
~stats.loc[:, ("snakemake_status", "solve_network")]

In [ ]:
shutil.path.es

In [ ]:
os.path.exists("/data/davidef/git_world/pypsa-earth/scenarios")

In [ ]:
os.path.isdir(f"/data/davidef/git_world/pypsa-earth/scenarios/{scenario}/")

In [ ]:
scenario = "ML"
os.path.isdir(os.path.abspath(os.path.join("/data/davidef/git_world", f"pypsa-earth/scenarios/{scenario}/")))

In [ ]:
stats[stats[("snakemake_status", "solve_network")] == False].index

In [ ]:
stats[("snakemake_status", "solve_network")]